In [6]:
from docx2python import docx2python
import numpy as np
import os
import json
import pandas as pd


Trade Unions (2010-2020, förbundspress):
    * A, LO centralt
    * Hotell- o resturangfacket;
    * H, Handelsanställdas förbund
    * L, Livsmedelsarbetarförbundet
    * K, Kommunalarbetarförbundet
    * DA,IF Metall
    * FF,Fastighetsansälldas förbund
    * BA,Byggnadsarbetarförbundet
    * E, Elektrikerförbundet
    * M, Målarna
    * S, Seko
    * T, Transportarbetarförbundet

### 1. Data Preparation 

In [29]:
## flatten irregular list
def flatten(xs): 
    res = []
    def loop(ys):
        for i in ys:
            if isinstance(i, list):
                loop(i)
            else:
                res.append(i)
    loop(xs)
    return res

# extract docx content
def extract(file):  
    '''file: in the format 'H - 2010 - Diskriminering.docx'
       return compact text as a string'''
        
    doc_result = flatten(docx2python(file).body )
    doc_list=[t for t in doc_result if t!='' and not str(t).__contains__('.png')
                           and not str(t).__contains__('.jpeg')
                           and not str(t).__contains__('.jpg')
                           and not str(t).__contains__('https://')
                           and not str(t).__contains__('<a href=')
                           and t!='URL: '
                           and not str(t).__contains__('avläst') ]
    return ' '.join(doc_list)

In [51]:
## go through directory and prepare text data set
path0='C:\\Users\\A550240\\Desktop\\LIU\\3.2_TextMining\\Project\\1.SelectedData'
unions=None # folers names under the directory
for pa,dir,fil in os.walk(path0):
    unions=dir
    break
unions.remove('.ipynb_checkpoints')
unions.remove('States')

files=None
#count=0, 483 files
content=[]
for u in unions:
    union=' '.join(str(u).split()[1:]) ## union name
    print(u)
    path1=os.path.join(path0,u)
    for pa,dir,fil in os.walk(path1):
        files=fil
        break
    for f in files:
        #count+=1
        f_split=str(f).split()
        year=[d for d in f_split if all(e.isnumeric() for e in d)][0] ## year of publishing
        #print(year)
        ind=[ind for ind,va in enumerate(f_split) if va=='-'] ## index for the last "-"
        title=' '.join(f_split[(ind[-1]+1):])[:-5] ## title of press
        path2=os.path.join(path1,f)
        doc=extract(path2)
        
        if len(doc)>0:
            new= {'union': union,
                  'year': year,
                  'title':title,
                  'press':doc}
        content.append(new)

## save as Json file
with open('sweunions.json', 'w', encoding='utf8') as json_file:  
    json.dump(content, json_file)

1. LO centralt
10. Elektrikerförbundet
11. Målarna
12. Seko
13. Transportarbetarförbundet
2. Hotell- o resturangfacket
3. Handelsanställdas förbund
4. Livsmedelsarbetarförbundet
5. Kommunalarbetarförbundet
6. IF Metall
8. Fastighetsansälldas förbund
9. Byggnadsarbetarförbundet


In [52]:
with open('sweunions.json','r',encoding='utf-8') as source: #C:/Users/A550240/Desktop/LIU/3.2_TextMining/Project/1.SelectedData/
    df = pd.read_json(source)

In [53]:
df.shape  ## shape of 482*4

(483, 4)

In [54]:
df['press'][125]

'Snabbspår för nyanlända Publicerad 31 Dec 2015 Inom kort kan ett snabbspår för nyanlända inom energibranschen vara i gång.På Åsbro kursgård ska en första grupp få sina kunskaper validerade och få kompletterande utbildning.– Det är bra. Det är en brist på montörer, säger Ulf Carlsson hos Elektrikerna. \xa0 \xa0 Det är många av nyanlända flyktingar som har en bakgrund i el- och energibranschen.För ett par månader sedan hade man lokaliserat 450 personer med bakgrund i energibranschen. Troligen har det kommit ett ytterligare antal sista tiden.Samtidigt pågår planer att man med ett snabbspår ska få så många av de nyanlända i arbete så fot som möjligt. \xa0 \xa0 Snabbspåret innebär\xa0att personer med en specifik utbildning kan få sin utbildning och kunskaper validerade. Det innebär att man ser över hur deras utbildning förhåller sig till svensk utbildning.Därefter ska en kompletteringsutbildning kunna sättas in.Man har träffats från de olika parterna för att diskutera detta. Det är Arbetsf

#### create dataframe for sentences

In [55]:
def find(string,char):
    return [i for i,k in enumerate(string) if k==char and string[i-1].isalpha()]        

In [62]:
content=[]
for i in range(483):
    doc=df['press'][i]
    ind=find(doc,'.')
    for key,va in enumerate(ind):
        if key==0:
            sen=doc[0:va]
        else:
            sen=doc[(ind[key-1]+1):va]
        new= {'union': df['union'][i],
              'year': int(df['year'][i]),
              'title':df['title'][i],
              'sentence':sen}
        content.append(new)

## save as Json file
with open('sweunions_sentences.json', 'w', encoding='utf8') as json_file:  
    json.dump(content, json_file)

In [63]:
len(content)

15395

In [64]:
content

[{'union': 'LO centralt',
  'year': 2010,
  'title': 'Arbetare utnyttjas',
  'sentence': 'Arbetare utnyttjas grovt  Arbetare utnyttjas grovt Erik Larsson 17 Dec 2010 Foto: Jessica Gow\xa0Bild: Jessica Gow Sverige kan räkna med att allt fler papperslösa kommer att exploateras på arbetsmarknaden i framtiden'},
 {'union': 'LO centralt',
  'year': 2010,
  'title': 'Arbetare utnyttjas',
  'sentence': ' Den bedömningen gör professor Gregor Noll på juridiska fakulteten, Lunds universitet'},
 {'union': 'LO centralt',
  'year': 2010,
  'title': 'Arbetare utnyttjas',
  'sentence': ' Han ser två tunga orsaker till att papperslösa i högre utsträckning riskerar att utnyttjas på Sveriges och hela EU:s arbetsmarknad'},
 {'union': 'LO centralt',
  'year': 2010,
  'title': 'Arbetare utnyttjas',
  'sentence': ' – Det ena är att vi allt mer konkurrerar med länders som åsidosätter arbetares rättigheter, säger han'},
 {'union': 'LO centralt',
  'year': 2010,
  'title': 'Arbetare utnyttjas',
  'sentence': '

### 2. Tokenizer

In [2]:
import stanza

#stanza.download('sv')

In [3]:
nlp=stanza.Pipeline(lang='sv',processors='tokenize,lemma')

2021-01-08 14:39:18 INFO: Loading these models for language: sv (Swedish):
| Processor | Package   |
-------------------------
| tokenize  | talbanken |
| lemma     | talbanken |

2021-01-08 14:39:18 INFO: Use device: cpu
2021-01-08 14:39:18 INFO: Loading: tokenize
2021-01-08 14:39:19 INFO: Loading: lemma
2021-01-08 14:39:19 INFO: Done loading processors!


In [4]:
## Tokenizer
def tokenizer(doc):
    tok=nlp(doc)
    re=[]
    stop=['.',':',',','i','att','med','på','en','ett','till','och','-','–','den',
          'vara','av','som','men','till','denna','för','in','(',')','eller',
          'sån','sedan','•','?','om','!','%','&','--','\'','*','**','"','”','“','[avläsT',
          '[',']','/','»','«',';','=','+',
          '[accessed','[aläst','[avläst','[online]','[sg_popup','[ur', '[vläst',
          '‚ä®sverige', '…','‰r','″', '→', '●', '\uf0b7']
    za1=['"','(',')','“','”','*',':',',','.']
    for i,j in enumerate(tok.sentences):
        for word in j.words:
            #print(word.lemma)
            if word.text in stop or word.text.isnumeric(): ## remove stop words and numbers
                pass            
            elif len(set(list('0123456789')).intersection(set(list(word.text))))>0: 
                pass            
            elif word.text not in za1 and len(set(za1).intersection(set(list(word.text))))>0:
                tem=str(word.text)
                for za in za1:
                    if tem.__contains__(za):
                        #print(tem)
                        ha=list(tem)
                        ha.remove(za)
                        tem=''.join(ha)
                f=[e.words for e in nlp(tem).sentences]
                if len(f)>0:
                    re.append(f[0][0].lemma)
            elif word.text.endswith('-') or word.text.startswith('-') and word.text!='-' and word.text!='--':
                #print(word.text)
                f=[e.words for e in nlp(word.text.replace('-','')).sentences]
                re.append(f[0][0].lemma)
            else:
                re.append(word.lemma)

        
    return re
## tokenizer('jag var ingejörer')
## ['jag', 'vara', 'ingejör']
## tokenizer(df['press'][6])

In [14]:
nlp(df['press'][125])

[]

Information Retrieval

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
vectorizer=TfidfVectorizer(tokenizer=tokenizer)
X=vectorizer.fit_transform(df['press']) ## fit vector space 
vocab=np.array(list(vectorizer.get_feature_names())) ## vocabulary
X.shape  ## 483*14516

(483, 14516)

In [17]:
list(vocab)

['(ata',
 '--',
 'Birén',
 'Borkers',
 'Cebb-sida',
 'Eu-direktiv',
 'Eu-projektet',
 'Eu-toppmöte',
 'a',
 'a-kassa',
 'a-kasse-grundande',
 'a-kasseavgift',
 'a-kasseavgiften',
 'a-kassegrundande',
 'a-kassetaket',
 'a-lag',
 'ab',
 'abadj',
 'abadji',
 'abdiker',
 'abdolaziz',
 'abdullah',
 'abf',
 'abf-hus',
 'absolut',
 'abstrakt',
 'absurd',
 'absurda',
 'absurt”att',
 'academedium',
 'accelererande',
 'acceptabel',
 'acceptabla',
 'acceptera',
 'ackholt',
 'ackmark',
 'ackord',
 'ackordsarbete',
 'ackordsavstämning',
 'ackordsfråga',
 'ackordsjobb',
 'ackordslön',
 'ackordssystem',
 'ad',
 'adhd',
 'administration',
 'administrationssynvinkel',
 'administrativ',
 'administrativa',
 'administrativt',
 'administrer',
 'administrera',
 'administreraenlig',
 'adna',
 'adress',
 'advokat',
 'af',
 'afasi',
 'affisch',
 'affär',
 'affärsidé',
 'affärslokal',
 'affärsupplägg',
 'affärsvärlden',
 'afghanistan',
 'afghansk',
 'afrika',
 'afs',
 'aftonblad',
 'aftonblader',
 'agenda',
 'a

In [18]:
len(vocab)

14516

In [19]:
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(metric = "cosine")
neigh.fit(X)


def search(query): ## find most 10 similar press documents
    query_vector = vectorizer.transform([query])
    idx = neigh.kneighbors(query_vector, n_neighbors = 10, return_distance=False)[0]
    
    return df.iloc[idx]

## search(df['press'][0])

In [20]:
search(df['press'][0])

,union,year,title,press
0,LO centralt,2010,Arbetare utnyttjas,Arbetare utnyttjas grovt Arbetare utnyttjas g...
365,Kommunalarbetarförbundet,2018,Papperslösa,”Papperslösa är mer utsatta” Fackligt center f...
249,Hotell- o resturangfacket,2018,Polisrazior,Polisrazzior splittrar facken Polisen ska få s...
315,Livsmedelsarbetarförbundet,2020,Fackligt center,Fackligt center hjälper de mest utsatta Nr 5/2...
451,Byggnadsarbetarförbundet,2014,Papperslösa,Papperslösa får inte medlemskap Papperslösa oc...
8,LO centralt,2010,Papperslösa- kolletkivavtal,Papperlösa får lön I nivå med kollektivavtalet...
74,LO centralt,2016,Regeringens förslag,Regeringens förslag om kontroll av papperslösa...
406,Fastighetsansälldas förbund,2015,Avtalsenlig lön,Rätt till avtalsenlig lönäven som papperslös M...
283,Livsmedelsarbetarförbundet,2011,Illegala invandrare,”Illegala invandrare kan bli legala” Nr 11/201...
285,Livsmedelsarbetarförbundet,2011,Migrationsverket,URL: ”Illegala invandrare kan bli legala” Nr 1...


In [21]:
def keywords(text, n=10):  ## find key words with highest tf-idf
    query_vector = vectorizer.transform([text])
    weight=query_vector.toarray()[0]
    ind=list(np.argsort(-1*weight)) ## sort idf in descending order

    return list(vocab[ind[:n]])

## print(keywords(df['press'][0]))

In [22]:
print(keywords(df['press'][0]))

['papperslösa', 'grega', 'noll', 'eus', 'gow', 'jessica', 'vara', 'han', 'grov', 'utanför']


### 3. Clustering 

In [23]:
# compute the Rand indices for the two clusterings
from itertools import combinations, permutations

def randind(k):
    cluster=KMeans(n_clusters=k,n_init=5).fit(reviews)
    lab=cluster.labels_  ## labels/class/cluster of each doc
    
    combs=list(combinations(list(range(483)),2))  ## all combinations of 2 documents
    a=0    ## times of clustering two comments into same group
    b=0    ## times of clustering two comments into different groups
    n_combs=len(combs) ## number of doc combinations
    
    for comb in combs:
        if df['category'][comb[0]]==df['category'][comb[1]] and list(lab)[comb[0]]==list(lab)[comb[1]]:
            a+=1
        if df['category'][comb[0]]!=df['category'][comb[1]] and list(lab)[comb[0]]!=list(lab)[comb[1]]:
            b+=1
    print(f"The Rand Index of {k}-class Kmeans clustering relative to original category is {(a+b)/n_combs}.")


In [24]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel


In [25]:
documents=[]
for i in range(483):
    print(i)
    documents.append(tokenizer(df['press'][i]))
common_dictionary= Dictionary(documents)
common_corpus = [common_dictionary.doc2bow(text) for text in documents]


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [26]:
k=3
model_cluster = LdaModel(common_corpus, num_topics=k)

In [27]:
model_cluster.print_topics()

[(0,
  '0.028*"37" + 0.028*"181" + 0.019*"72" + 0.016*"46" + 0.012*"89" + 0.010*"145" + 0.010*"182" + 0.008*"97" + 0.006*"60" + 0.006*"155"'),
 (1,
  '0.028*"37" + 0.025*"181" + 0.024*"46" + 0.018*"72" + 0.015*"89" + 0.015*"182" + 0.013*"145" + 0.010*"60" + 0.009*"155" + 0.007*"97"'),
 (2,
  '0.033*"37" + 0.032*"181" + 0.021*"72" + 0.018*"46" + 0.014*"145" + 0.012*"182" + 0.011*"89" + 0.010*"60" + 0.009*"97" + 0.008*"155"')]

In [28]:
vocab_cluster=list(common_dictionary.values())
for i in range(k):
    print(f"Here come most significant 10 terms for cluster {i}:")
    print(f"{[vocab_cluster[fre[0]] for fre in model_cluster.get_topic_terms(i)]}")

Here come most significant 10 terms for cluster 0:
['den', 'vara', 'ha', 'en', 'inte', 'skola', 'vi', 'kunna', 'få', 'säga']
Here come most significant 10 terms for cluster 1:
['den', 'vara', 'en', 'ha', 'inte', 'vi', 'skola', 'få', 'säga', 'kunna']
Here come most significant 10 terms for cluster 2:
['den', 'vara', 'ha', 'en', 'skola', 'vi', 'inte', 'få', 'kunna', 'säga']


### 4. Embeddings 

### Save/Load 

In [12]:
import pickle

## df, data frame of "sweunions.json" 
## X, fitted tf-idf matrix
## vocab, vocabulary of tf-idf matrix
## vectorizer, vectorizer with all fitted parameters
## neigh, fitted nearest neighbor space
## vocab_cluster, vocabulary of clusterings
## model_cluster, model of clusterings

## tokenizer(), tokenize documents
## search(), find most 10 similar press documents
## keywords(), find key words with highest tf-idf


In [38]:
#pickle.dump([df,X,vocab,vectorizer,neigh,vocab_cluster,model_cluster], open( "./States/code0_vars.p", "wb" ) ) ## save
pickle.dump(vocab, open( "./States/code0_vocab.p", "wb" ) ) ## save

In [30]:
[df,X,vocab,vectorizer,neigh,vocab_cluster,model_cluster]= pickle.load( open( "./States/code0_vars.p", "rb" ) ) ## load session

In [37]:
model_cluster